In [1]:
from os import system
import os
import pandas as pd
import numpy as np
import time
import copy
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import graphviz
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score

# Data Preprocessing (No need to use in case of pickle load)

In [2]:
# data1 = pd.read_csv('flight.csv')
# data2 = pd.read_csv('flight_neu.csv')
# data3 = pd.read_csv('flight_wn.csv')
# # data3 = pd.read_csv('dataset/flight_mustafa.csv')
# # data4 = pd.read_csv('dataset/flight_ubuntu.csv')
# frames = [data1, data2, data3]#, data4]
# data = pd.concat(frames)
# N_B1 = np.sum(data["controller_ID"] == "Backstepping_1")
# N_B2 = np.sum(data["controller_ID"] == "Backstepping_2")
# N_B3 = np.sum(data["controller_ID"] == "Backstepping_3")
# N_B4 = np.sum(data["controller_ID"] == "Backstepping_4")

# print("Dataset size: ", data.shape)
# print("Backstepping_1 (the most agile): %", N_B1/(N_B1+N_B2+N_B3+N_B4)*100)
# print("Backstepping_2 (agile): %", N_B2/(N_B1+N_B2+N_B3+N_B4)*100)
# print("Backstepping_3 (smooth): %", N_B3/(N_B1+N_B2+N_B3+N_B4)*100)
# print("Backstepping_4 (the smoothest): %", N_B4/(N_B1+N_B2+N_B3+N_B4)*100)
# data.head(10)

In [3]:
data1 = pd.read_csv('flight_gauss.csv')
# data2 = pd.read_csv('flight_neu.csv')
# data3 = pd.read_csv('flight.csv')
# data4 = pd.read_csv('flight_wn.csv')
frames = [data1]#, data2, data3, data4]
data = pd.concat(frames)
N_B1 = np.sum(data["controller_ID"] == "Backstepping_1")
N_B2 = np.sum(data["controller_ID"] == "Backstepping_2")
N_B3 = np.sum(data["controller_ID"] == "Backstepping_3")
N_B4 = np.sum(data["controller_ID"] == "Backstepping_4")

print("Dataset size: ", data.shape)
print("Backstepping_1 (the most agile): %", N_B1/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_2 (agile): %", N_B2/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_3 (smooth): %", N_B3/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_4 (the smoothest): %", N_B4/(N_B1+N_B2+N_B3+N_B4)*100)
data.head(10)

Dataset size:  (76292, 38)
Backstepping_1 (the most agile): % 33.78598018140828
Backstepping_2 (agile): % 3.076338279242909
Backstepping_3 (smooth): % 33.60902846956431
Backstepping_4 (the smoothest): % 29.52865306978451


,x0,y0,z0,x_dot0,y_dot0,z_dot0,phi0,theta0,yaw0,phi_dot0,...,yp,zp,x_dotp,y_dotp,z_dotp,x_ddotp,y_ddotp,z_ddotp,u_abs_p,controller_ID
0,1.850561,1.359698,4.642320,-0.582726,-0.762233,1.538706,-0.915301,0.180243,-1.502666,0.449425,...,1.360460,4.640781,-0.595883,-0.761163,1.546005,1.280145,-0.520413,0.632062,1.000000,Backstepping_4
1,1.841616,1.344309,4.673153,-0.324584,-0.773924,1.549486,-0.771842,0.167495,-1.451927,10.473490,...,1.345901,4.670269,-0.572248,-0.771392,1.557965,1.207893,-0.556212,0.626819,115.216348,Backstepping_1
2,1.836987,1.328816,4.704266,-0.151595,-0.775964,1.567250,-0.542689,0.157239,-1.351921,12.101720,...,1.330360,4.701553,-0.548841,-0.782887,1.570446,1.133008,-0.593112,0.621245,98.965678,Backstepping_1
3,1.835076,1.313213,4.735675,-0.049155,-0.784552,1.569280,-0.296302,0.155773,-1.226957,12.394616,...,1.314581,4.733086,-0.526920,-0.795112,1.582814,1.059316,-0.629213,0.615614,143.696512,Backstepping_1
4,1.834670,1.297282,4.767187,0.000199,-0.807563,1.581925,-0.053117,0.143138,-1.092890,11.735078,...,1.298551,4.764865,-0.506460,-0.808050,1.595070,0.986810,-0.664521,0.609928,201.414022,Backstepping_1
5,1.834775,1.280825,4.798924,0.003299,-0.833805,1.607577,0.159959,0.092304,-0.959613,8.793254,...,1.282254,4.796888,-0.487439,-0.821687,1.607211,0.915484,-0.699041,0.604187,141.531459,Backstepping_3
6,1.834578,1.263716,4.831460,-0.027346,-0.875931,1.652156,0.292072,0.027469,-0.830722,4.549058,...,1.265679,4.829153,-0.469833,-0.836007,1.619237,0.845330,-0.732779,0.598393,94.206820,Backstepping_3
7,1.833550,1.245749,4.864548,-0.077373,-0.913907,1.672903,0.349325,-0.032035,-0.706248,1.394529,...,1.248810,4.861657,-0.453619,-0.850993,1.631146,0.776341,-0.765739,0.592545,90.532910,Backstepping_3
8,1.831458,1.226965,4.898019,-0.132104,-0.955076,1.678926,0.355552,-0.075333,-0.586113,-0.608326,...,1.231635,4.894398,-0.438772,-0.866631,1.642938,0.708510,-0.797927,0.586644,76.831918,Backstepping_3
9,1.828265,1.207211,4.931433,-0.186892,-1.015663,1.652699,0.330217,-0.102366,-0.470233,-1.822644,...,1.214140,4.927374,-0.425270,-0.882905,1.654612,0.641831,-0.829347,0.580691,62.791172,Backstepping_3


In [4]:
N_Test = 5000
dataset = data.values
controller_labels = {'Backstepping_1': 0, 'Backstepping_2': 1, 'Backstepping_3': 2, 'Backstepping_4': 3}
np.random.shuffle(dataset)
y = np.array([controller_labels[data[-1]] for data in dataset]).reshape(-1,)
X = dataset[:,:-1]

X_test = X[0:N_Test,:]
y_test = y[0:N_Test]

X_train, X_val, y_train, y_val = train_test_split(X[N_Test:,:], y[N_Test:], test_size=0.1, random_state=42)

# Saving the objects:
# with open('dataset.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([X_train, X_val, X_test, y_train, y_val, y_test], f)
    
pickle.dump([X_train, X_val, X_test, y_train, y_val, y_test], open("dataset.pkl","wb"), protocol=2)


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print ("X_train size: ",X_train.shape)
print ("X_val size: ",X_val.shape)
print ("X_test size: ",X_test.shape)

X_train size:  (64162, 37)
X_val size:  (7130, 37)
X_test size:  (5000, 37)


# Data Loading and Merging

In [5]:
with open('dataset.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    X_train, X_val, X_test, y_train, y_val, y_test = pickle.load(f)

#with open('dataset_mustafa.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
#    X_trainM, X_valM, X_testM, y_trainM, y_valM, y_testM = pickle.load(f)
    
#Xtrain = np.concatenate((X_train, X_trainM), axis=0)
#Xval = np.concatenate((X_val, X_valM), axis=0)
#Xtest = np.concatenate((X_test, X_testM), axis=0)

#ytrain = np.concatenate((y_train, y_trainM), axis=0)
#yval = np.concatenate((y_val, y_valM), axis=0)
#ytest = np.concatenate((y_test, y_testM), axis=0)

Xtrain = X_train
Xval = X_val
Xtest = X_test
ytrain = y_train
yval = y_val
ytest = y_test


scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

print ("Xtrain size: ",Xtrain.shape)
print ("Xval size: ",Xval.shape)
print ("Xtest size: ",Xtest.shape)

Xtrain size:  (64162, 37)
Xval size:  (7130, 37)
Xtest size:  (5000, 37)


# Decision Tree Design

In [6]:
from sklearn.tree._tree import TREE_LEAF

def prune_index(inner_tree, index, threshold):
    if inner_tree.value[index].min() < threshold:
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
    # if there are shildren, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_index(inner_tree, inner_tree.children_left[index], threshold)
        prune_index(inner_tree, inner_tree.children_right[index], threshold)


feature_names = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
                 'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
                 'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
                 'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p', 'wind_direct']

class_names = ['Backstepping_1', 'Backstepping_2', 'Backstepping_3', 'Backstepping_4']
clf = DecisionTreeClassifier()
clf = clf.fit(Xtrain, ytrain)
# prune_index(clf.tree_, 0, 10)

train_pred = clf.predict(Xtrain)
val_pred = clf.predict(Xval)
test_pred = clf.predict(Xtest)


train_acc = accuracy_score(ytrain, train_pred)
val_acc = accuracy_score(yval, val_pred)
test_acc = accuracy_score(ytest, test_pred)

print ("Decision Tree Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))


forest = RandomForestClassifier(bootstrap=True, n_estimators=100, 
                                min_samples_split=12, min_samples_leaf=5, max_features=10, max_depth=80)
forest = forest.fit(X_train, y_train)

train_pred = forest.predict(X_train)
val_pred = forest.predict(X_val)
test_pred = forest.predict(X_test)


train_acc = accuracy_score(y_train, train_pred)
val_acc = accuracy_score(y_val, val_pred)
test_acc = accuracy_score(y_test, test_pred)

print ("Random Forest Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))

# dot_data = tree.export_graphviz(clf, out_file=None, 
#                       feature_names=feature_names,  
#                       class_names=class_names,  
#                       filled=True, rounded=True,  
#                       special_characters=True)
# graph = graphviz.Source(dot_data) 
# graph

Decision Tree Train Acc: 1.0, Val Acc: 0.627, Test Acc: 0.609
Random Forest Train Acc: 0.911, Val Acc: 0.728, Test Acc: 0.717


# Neural Network Design

In [7]:
stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
stats_filename = 'params_results.csv'

def write_results(results): 
    df_stats = pd.DataFrame([results], columns=stats_columns)
    df_stats.to_csv(stats_filename, mode='a', index=False,header=not os.path.isfile(stats_filename))

def predict(X, y, model):
    #Validation part
    model.eval()  # Set model to training mode
    
    inputs, labels = torch.from_numpy(X).to(device), torch.from_numpy(y).to(device).long()

    outputs = model(inputs.float())
    _, preds = torch.max(outputs, 1)

    loss = criterion(outputs, labels)
    
    accuracy = torch.sum(preds == labels.data).item() / float(inputs.size(0))
    
    print ("Test data, Loss: {0:.3}, Accuracy: {1:.4}".format(loss.item(), accuracy))
    

def shuffle_dataset(X, y):
    p = np.random.permutation(len(X))
    return X[p], y[p]

def train_model(X, y, X_val, y_val, model, criterion, optimizer, scheduler, minibatch_size, num_epochs=25):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    losses_train = []
    losses_val = []
    accuracy_train = []
    accuracy_val = []
    # path = F"/content/drive/My Drive/best_model.pt"
#     directory = path_name

#     if not os.path.exists(directory):
#         os.makedirs(directory)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        X_train, y_train = shuffle_dataset(X, y)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        losses_iter = []
        accuracy_iter = []

        # Iterate over data.
        for i in range(0, X_train.shape[0], minibatch_size):
            # Get pair of (X, y) of the current minibatch/chunk             
            X_batch = X_train[i:i + minibatch_size]
            y_batch = y_train[i:i + minibatch_size]

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = torch.from_numpy(X_batch).to(device), torch.from_numpy(y_batch).to(device).long()


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs.float())
                _, preds = torch.max(outputs, 1)
                
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()
            
            
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # print ("losses_iter", loss.item() * inputs.size(0))
            # print ("accuracy_iter", torch.sum(preds == labels.data).item() / float(inputs.size(0)))

            losses_iter.append(loss.item())
            accuracy_iter.append(torch.sum(preds == labels.data).item() / float(inputs.size(0)))
        
        
        train_loss = np.mean(losses_iter)
        train_acc = np.mean(accuracy_iter)

        losses_train.append(train_loss)
        accuracy_train.append(train_acc)


        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        #Validation part
        model.eval()  # Set model to training mode
        
        inputs, labels = torch.from_numpy(X_val).to(device), torch.from_numpy(y_val).to(device).long()
        
        optimizer.zero_grad()
        
        outputs = model(inputs.float())
        _, preds = torch.max(outputs, 1)

        loss = criterion(outputs, labels)
        
        val_loss = loss.item()
        scheduler.step(val_loss)
        
        val_acc = torch.sum(preds == labels.data).item() / float(inputs.size(0))
        
        losses_val.append(val_loss)
        accuracy_val.append(val_acc)
        
        
        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

#         deep copy the model
        if val_acc > best_acc:
            best_train_acc = train_acc
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            path = "best_model.pt"
            torch.save(best_model_wts, path)

        print()


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}s'.format(time_elapsed))
    print('Best Train Acc: {:4f}'.format(best_train_acc))
    print('Best Val Acc: {:4f}'.format(best_val_acc))
    
    
#     stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
    layers = [module for module in model.modules() if type(module) != nn.Sequential]
    #write_results([layers, num_epochs, minibatch_size, learning_rate, optimizer.state_dict, scheduler.state_dict, best_train_acc, best_val_acc])

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.0)
        self.fc1 = nn.Linear(37, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc4(x))
        x = self.drop_layer(x)
        x = self.fc5(x)
        return x

n_epochs = 15
minibatch_size = 32
learning_rate = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model = Net()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
final_model = train_model(Xtrain, ytrain, Xval, yval, model, criterion, optimizer, scheduler, minibatch_size, num_epochs=n_epochs)

Epoch 0/14
----------
Training Loss: 1.1416 Acc: 0.4292
Validation Loss: 1.1191 Acc: 0.4546

Epoch 1/14
----------
Training Loss: 1.1023 Acc: 0.4622
Validation Loss: 1.1095 Acc: 0.4523

Epoch 2/14
----------
Training Loss: 1.0757 Acc: 0.4835
Validation Loss: 1.0657 Acc: 0.4987

Epoch 3/14
----------
Training Loss: 1.0330 Acc: 0.5215
Validation Loss: 1.0625 Acc: 0.5205

Epoch 4/14
----------
Training Loss: 1.0122 Acc: 0.5390
Validation Loss: 1.0540 Acc: 0.5180

Epoch 5/14
----------
Training Loss: 0.9987 Acc: 0.5449
Validation Loss: 1.0091 Acc: 0.5449

Epoch 6/14
----------
Training Loss: 0.9900 Acc: 0.5495
Validation Loss: 1.0142 Acc: 0.5362

Epoch 7/14
----------
Training Loss: 0.9841 Acc: 0.5534
Validation Loss: 1.0085 Acc: 0.5440

Epoch 8/14
----------
Training Loss: 0.9759 Acc: 0.5582
Validation Loss: 1.0031 Acc: 0.5478

Epoch 9/14
----------
Training Loss: 0.9714 Acc: 0.5620
Validation Loss: 0.9887 Acc: 0.5537

Epoch 10/14
----------
Training Loss: 0.9656 Acc: 0.5632
Validation Lo

In [9]:
predict(Xtest, ytest, final_model)

Test data, Loss: 0.971, Accuracy: 0.5502


In [10]:
# torch.cuda.empty_cache()

# with open('dataset_mustafa.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
#     X_trainM, X_valM, X_testM, y_trainM, y_valM, y_testM = pickle.load(f)
    
# scalerM = StandardScaler()
# scalerM.fit(X_trainM)

# X_trainM = scalerM.transform(X_trainM)
# X_valM = scalerM.transform(X_valM)
# X_testM = scalerM.transform(X_testM)

In [11]:
# for i in range(1000):
#     torch.cuda.empty_cache()

# predict(X_valM, y_valM, final_model)

In [12]:
# val_predM = forest.predict(X_valM)
# test_predM = forest.predict(X_testM)

# val_accM = accuracy_score(y_valM, val_predM)
# test_accM = accuracy_score(y_testM, test_predM)

# print ("Random Forest, Val Acc: {0:.3}, Test Acc: {1:.3}".format(val_accM, test_accM))

# SVM

In [13]:
# svc = svm.SVC(kernel='rbf')
# svc.fit(X_train, y_train)

# svc = svc.fit(Xtrain, ytrain)

# train_pred = svc.predict(Xtrain)
# val_pred = svc.predict(Xval)
# test_pred = svc.predict(X_est)


# train_acc = accuracy_score(ytrain, train_pred)
# val_acc = accuracy_score(yval, val_pred)
# test_acc = accuracy_score(ytest, test_pred)

# print ("SVM Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))